In [11]:
import os

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import matplotlib

import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client('sagemaker')

In [2]:
dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip'
s3_prefix = 'aws-machine-learning-specialty/algorithms/xgboost/classification'
s3_bucket = sagemaker_session.default_bucket()

#### Download Dataset

In [3]:
!wget -P . $dataset_url 
local_filename = dataset_url.split('/')[-1]
!unzip $local_filename
!rm $local_filename

--2021-02-02 17:41:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444572 (434K) [application/x-httpd-php]
Saving to: ‘./bank-additional.zip’

bank-additional.zip 100%[===================>] 434.15K  1.27MB/s    in 0.3s    

2021-02-02 17:41:47 (1.27 MB/s) - ‘./bank-additional.zip’ saved [444572/444572]

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/bank-additional/
  inflating: __MACOSX/bank-additional/._.DS_Store  
  inflating: bank-additional/.Rhistory  
  inflating: bank-additional/bank-additional-full.csv  
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: __MAC

In [3]:
raw_data = pd.read_csv('bank-additional/bank-additional-full.csv',delimiter=";")
raw_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### One hot encoding

In [4]:
model_data = pd.get_dummies(raw_data)
model_data.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0


### Randomize the data

In [6]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.8 * len(model_data))])
print(train_data.shape, test_data.shape)

(32950, 65) (8238, 65)


### Preparación
Inicialmente para el modelo removemos el header y formateamos para que la primera columna sea el target

In [8]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

In [9]:
!head train.csv

0,54,99,3,999,0,-1.1,94.601,-49.5,0.987,4963.6,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
0,56,143,2,999,0,1.4,94.465,-41.8,4.967,5228.1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
0,32,120,2,999,0,-1.8,92.89299999999999,-46.2,1.3130000000000002,5099.1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
0,46,134,3,999,0,1.1,93.994,-36.4,4.8580000000000005,5191.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
0,35,34,2,999,0,1.4,94.465,-41.8,4.967,5228.1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
0,34,1576,3,999,0,1.4,94.465,-41.8,4.865,5228.1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
0,30,213,3,999,0,1.4,94.465,-41.8,4.864,5228

### Subida a s3

In [13]:
boto3.Session().resource('s3').Bucket(s3_bucket).Object(os.path.join(s3_prefix, 'train/train.csv')).upload_file('train.csv')
# https://sagemaker.readthedocs.io/en/stable/api/utility/inputs.html
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(s3_bucket, s3_prefix), content_type='csv')

### Creacion de una instancia del estimador XGBoost y sus hiperparámetros

In [24]:
#!pip install -U sagemaker

In [23]:
xgboost_container = sagemaker.image_uris.retrieve('xgboost', boto3.Session().region_name, version='latest')
xgb = sagemaker.estimator.Estimator(xgboost_container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(s3_bucket, s3_prefix),
                                    sagemaker_session=sagemaker_session)

xgb.set_hyperparameters(eta=0.1,objective='binary:logistic',num_round=25)

### Train Model

In [25]:
xgb.fit({'train': s3_input_train})

2021-02-02 18:28:31 Starting - Starting the training job...
2021-02-02 18:28:34 Starting - Launching requested ML instancesProfilerReport-1612290511: InProgress
......
2021-02-02 18:30:02 Starting - Preparing the instances for training......
2021-02-02 18:31:03 Downloading - Downloading input data...
2021-02-02 18:31:23 Training - Downloading the training image..Arguments: train
[2021-02-02:18:31:41:INFO] Running standalone xgboost training.
[2021-02-02:18:31:41:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-02-02:18:31:41:INFO] File size need to be processed in the node: 4.98mb. Available memory size in the node: 8424.43mb
[2021-02-02:18:31:41:INFO] Determined delimiter of CSV input is ','
[18:31:41] S3DistributionType set as FullyReplicated
[18:31:41] 32950x63 matrix with 2075850 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[18:31:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_dep

### Deploy Model

In [26]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

----------------!

### Realizando Predicciones

In [28]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer

# Predict whether bank customers in the test dataset will subscribe to a term deposit
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
#xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(8238,)


### Evaluación

In [29]:
# Evaluate the performance and accuracy of the model
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "Not Subscribed", "Subscribed"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("Not Subscribed", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Subscribed", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 91.8%

Predicted      Not Subscribed Subscribed
Observed
Not Subscribed 94% (7043)    31% (227)
Subscribed      6% (452)     69% (516) 



In [30]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint_name)